# Ramsey King
# DSC 550 - Data Mining
# October 10, 2021
# Term Project Milestone 1

In [4]:
# Retreive talks from topics page using beautifulsoup

from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [25]:
# Get list of urls for each of the topics

url = "https://speeches.byu.edu/topics/"

url_list = ['life/', 'jesus-christ/', 'faith/', 'learning/', 'service/']

links = []
new_links = []
for u in url_list:

    response = requests.get(url+u)
    html = response.content
    video = '?M=V'
    soup = bs(html, "lxml")
    for link in soup.findAll("a", attrs={'href': re.compile("^https://speeches.byu.edu/talks/[\D+]")}):
        links.append(link.get('href').replace(video, ''))

    audio = '?M=A'
    for link in links:
        new_link = link.replace(audio, '')
        new_links.append(new_link)
    new_links = list(set(new_links))



In [69]:
# Retrieves talk text from each of the urls using beautiful soup.
talk_text = []
title_text = []
topics = []
dates = []
speakers = [] # showing 458 speakers vs 456 talks
positions = [] # showing 300 or so vs 456 talks

for link in new_links:
    url = link
    response = requests.get(url)
    html = response.content
    soup = bs(html, 'lxml')
    temp_list = []
    for div in soup.find_all('div', class_='single-speech__content'):
        talk_text.append(div.get_text(strip=True))
    for h1 in soup.find_all('h1', class_='single-speech__title'):
        title_text.append(h1.get_text(strip=True))
    '''for h2 in soup.find_all('h2', class_='single-speech__speaker'):
        speakers.append(h2.get_text(strip=True))
        print(speakers[-1])'''
    '''for p in soup.find_all('p', class_='single-speech__speaker-subtext single-speech__speaker-position'):
        positions.append(p.get_text(strip=True) if p else "No Position Found")
        # positions.append(p.get_text(strip=True))
        print(positions[-1])'''
    for p in soup.find_all('p', class_='single-speech__speaker-subtext single-speech__date'):
        dates.append(p.get_text(strip=True))
    for p in soup.find_all('p', class_='single-speech__related-tag'):
        temp_list.append(p.get_text(strip=True))
        temp_str = ','.join(temp_list)
    topics.append(temp_str)
    # print('topics:', temp_str)


# dict = {'titles': title_text , 'speaker': speakers,'position': positions, 'date': dates, 'talks': talk_text, 'topics': topics, }
dict = {'titles': title_text , 'date': dates, 'talks': talk_text, 'topics': topics, }
# print('title length: ', len(title_text), '|talk length: ', len(talk_text), '|topic length: ', len(topics), '|speaker length:', len(speakers), '|positions length:', len(positions), '|date length:', len(dates))
df = pd.DataFrame(dict)
df.to_csv('talks.csv')
# talk_text[1]

In [70]:
df.head(10)

,titles,date,talks,topics
0,A Proper Orientation on the Threshold of Life,"January 1, 1962","The things of the world, or of the flesh, must...","Life,Revelation"
1,Building Your Future,"October 3, 2000",It is frightening to be asked to speak to you ...,"Learning,Life"
2,Would You Sell?,"March 13, 1984",It is reported that the Prophet Joseph Smith s...,"Life,Opportunity"
3,“One Step Enough”,"June 30, 1992",In a Facetious Little Essay entitled “On Trans...,Faith
4,The Story of My Life,"October 30, 1951",Thomas L. Martin humorously describes the main...,"Adversity,Attitude,Blessings,Commitment,Decisi..."
5,The Power of Your Words,"January 31, 2017",I would like to explain the sequence of how I ...,"Faith,Fear,Podcast: Recent Speeches"
6,Prepare for Useful Service,"December 3, 1963",Antoine R. Ivins gives students spiritual and ...,"Preparation,Service"
7,Learning by Heart,"August 12, 2004",It is my privilege to be able to say a few wor...,Learning
8,To Establish a Secure Foundation for Life,"March 18, 2008",I rejoice in the singular privilege of being o...,"Happiness,Life,Righteousness"
9,On the Measurements We Make in Life,"November 12, 2013",I am grateful for the opportunity to be here w...,"Judgement,Life,Weaknesses,Podcast: By Study an..."


In [76]:
# get character lengths of each talk (may need to eliminate those less than 250 chars)
df['char_lengths'] = pd.Series([len(x) for x in df.talks])
df.head(10)

,titles,date,talks,topics,char_lengths
0,A Proper Orientation on the Threshold of Life,"January 1, 1962","The things of the world, or of the flesh, must...","Life,Revelation",236
1,Building Your Future,"October 3, 2000",It is frightening to be asked to speak to you ...,"Learning,Life",23750
2,Would You Sell?,"March 13, 1984",It is reported that the Prophet Joseph Smith s...,"Life,Opportunity",25098
3,“One Step Enough”,"June 30, 1992",In a Facetious Little Essay entitled “On Trans...,Faith,26039
4,The Story of My Life,"October 30, 1951",Thomas L. Martin humorously describes the main...,"Adversity,Attitude,Blessings,Commitment,Decisi...",239
5,The Power of Your Words,"January 31, 2017",I would like to explain the sequence of how I ...,"Faith,Fear,Podcast: Recent Speeches",17937
6,Prepare for Useful Service,"December 3, 1963",Antoine R. Ivins gives students spiritual and ...,"Preparation,Service",216
7,Learning by Heart,"August 12, 2004",It is my privilege to be able to say a few wor...,Learning,15538
8,To Establish a Secure Foundation for Life,"March 18, 2008",I rejoice in the singular privilege of being o...,"Happiness,Life,Righteousness",15520
9,On the Measurements We Make in Life,"November 12, 2013",I am grateful for the opportunity to be here w...,"Judgement,Life,Weaknesses,Podcast: By Study an...",18412


In [92]:
# get word count of each talk
df['word_count'] = df['talks'].apply(lambda x: len(re.findall(r'\w+', x)))
df.head(10)

,titles,date,talks,topics,char_lengths,word_count
0,A Proper Orientation on the Threshold of Life,"January 1, 1962","The things of the world, or of the flesh, must...","Life,Revelation",236,43
1,Building Your Future,"October 3, 2000",It is frightening to be asked to speak to you ...,"Learning,Life",23750,4084
2,Would You Sell?,"March 13, 1984",It is reported that the Prophet Joseph Smith s...,"Life,Opportunity",25098,4579
3,“One Step Enough”,"June 30, 1992",In a Facetious Little Essay entitled “On Trans...,Faith,26039,4710
4,The Story of My Life,"October 30, 1951",Thomas L. Martin humorously describes the main...,"Adversity,Attitude,Blessings,Commitment,Decisi...",239,41
5,The Power of Your Words,"January 31, 2017",I would like to explain the sequence of how I ...,"Faith,Fear,Podcast: Recent Speeches",17937,3326
6,Prepare for Useful Service,"December 3, 1963",Antoine R. Ivins gives students spiritual and ...,"Preparation,Service",216,34
7,Learning by Heart,"August 12, 2004",It is my privilege to be able to say a few wor...,Learning,15538,2775
8,To Establish a Secure Foundation for Life,"March 18, 2008",I rejoice in the singular privilege of being o...,"Happiness,Life,Righteousness",15520,2676
9,On the Measurements We Make in Life,"November 12, 2013",I am grateful for the opportunity to be here w...,"Judgement,Life,Weaknesses,Podcast: By Study an...",18412,3264


In [95]:
# reduce the topic to the ones selected from our original selection ('life', 'jesus-christ', 'faith', 'learning', 'service')

topic_selection_list = [x[:-1] for x in url_list]

# df['topics_2'] = df['topics'].apply(lambda x: x for x in x in topic_selection_list)


# df.head(10)
